In [ ]:
!pip install anndata

Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 5, in <module>
    from pip._internal.cli.main import main
ModuleNotFoundError: No module named 'pip'


In [ ]:
!pip install scanpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 25.2 MB/s 
     |████████████████████████████████| 9.4 MB 36.4 MB/s 
     |████████████████████████████████| 88 kB 8.3 MB/s 
     |████████████████████████████████| 965 kB 48.7 MB/s 
     |████████████████████████████████| 295 kB 83.3 MB/s 
     |████████████████████████████████| 1.1 MB 76.6 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=4e649f4eb7b8d978dc1d87287003c4469fbe41a654a7b1749e78906bdc009819
  Stored in directory: /root/.cache/pip/wheels/a9/3a/67/06a8950e053725912e6a8c42c4a3a241410f6487b8402542ea
  Created wheel for pynndescent: filename=pynndescent-0.5.8-py3-none-any.whl size=55513 sha256=64c861d16723351cd00df7248e0319604728b335af6ab2f5fc8e8ff16e93a233
  Stored in directory: /root/.cache/pip/wheels/1c/63/3a/29954bca1a27ba100e

In [ ]:
!pip install hdf5plugin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 16.9 MB 20.7 MB/s 


In [ ]:
!pip install scikit-misc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.8 MB 29.2 MB/s 


In [ ]:
# Import required packages
import os
import h5py
import hdf5plugin
import shutil

import pandas as pd
import anndata as ad
import numpy as np
import scanpy as sc

from google.colab import drive
from scipy.sparse import csc_matrix

In [ ]:
# Mount drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Declare useful constants
DATA_DIR = '/content/drive/MyDrive/Thesis/dance/dance/data'
INPUT_SUBTASK_NAME = 'openproblems_competition_cite_raw_rna'
OUTPUT_SUBTASK_NAME = 'openproblems_competition_cite_fltr_raw_rna'
RANDOM_SEED = 123

In [ ]:
assert os.path.exists(os.path.join(DATA_DIR, INPUT_SUBTASK_NAME))

In [ ]:
# Read h5ad files containing raw GEX counts
input_train_mod1 = ad.read_h5ad(os.path.join(DATA_DIR, 
                                             INPUT_SUBTASK_NAME, 
                                             f'{INPUT_SUBTASK_NAME}.censor_dataset.output_train_mod1.h5ad'))

input_test_mod1 = ad.read_h5ad(os.path.join(DATA_DIR, 
                                            INPUT_SUBTASK_NAME, 
                                            f'{INPUT_SUBTASK_NAME}.censor_dataset.output_test_mod1.h5ad'))

In [ ]:
# Concatenate the train and test data to determine highly variable genes across
# both
input_mod1 = ad.concat([input_train_mod1, input_test_mod1])

In [ ]:
# Rename the day obs column to batch to ensure compatibility with DANCE
input_mod1.obs.columns = ['batch', 'donor', 'cell_type']

In [ ]:
# Add feature_type columns to ensure compatibility with DANCE
input_mod1.var['feature_types'] = 'GEX'

In [ ]:
# Perform the highly variable genes calculation
sc.pp.highly_variable_genes(input_mod1,
                            flavor='seurat_v3', 
                            n_top_genes=10000, 
                            batch_key='batch')

In [ ]:
# Split the data again into the output and test data
output_train_mod1 = input_mod1[:input_train_mod1.shape[0] , input_mod1.var['highly_variable'] == True]
output_test_mod1 = input_mod1[input_train_mod1.shape[0]: , input_mod1.var['highly_variable'] == True]

In [ ]:
# Convert to CSC matrices
output_train_mod1_sparse = csc_matrix(output_train_mod1.X)
output_test_mod1_sparse = csc_matrix(output_test_mod1.X)

In [ ]:
output_train_mod1.X = None
output_test_mod1.X = None
output_train_mod1.X = output_train_mod1_sparse
output_test_mod1.X = output_test_mod1_sparse

In [ ]:
# Make the output directory and write the filtered GEX datasets
os.makedirs(os.path.join(DATA_DIR, OUTPUT_SUBTASK_NAME), exist_ok=True)

output_train_mod1.write(os.path.join(DATA_DIR,
                                     OUTPUT_SUBTASK_NAME, 
                                     f'{OUTPUT_SUBTASK_NAME}.censor_dataset.output_train_mod1.h5ad'))

output_test_mod1.write(os.path.join(DATA_DIR, 
                                    OUTPUT_SUBTASK_NAME, 
                                    f'{OUTPUT_SUBTASK_NAME}.censor_dataset.output_test_mod1.h5ad'))

In [ ]:
# Read ADT datasets
input_train_mod2 = ad.read_h5ad(os.path.join(DATA_DIR, 
                                             INPUT_SUBTASK_NAME, 
                                             f'{INPUT_SUBTASK_NAME}.censor_dataset.output_train_mod2.h5ad'))

input_test_mod2 = ad.read_h5ad(os.path.join(DATA_DIR, 
                                            INPUT_SUBTASK_NAME, 
                                            f'{INPUT_SUBTASK_NAME}.censor_dataset.output_test_mod2.h5ad'))

In [ ]:
# Rename the day obs column to batch to ensure compatibility with DANCE
input_train_mod2.obs.columns = ['batch', 'donor', 'cell_type']
input_test_mod2.obs.columns = ['batch', 'donor', 'cell_type']

In [ ]:
# Add feature_type columns to ensure compatibility with DANCE
input_train_mod2.var['feature_types'] = 'ADT'
input_test_mod2.var['feature_types'] = 'ADT'

In [ ]:
# Convert to CSC matrices
input_train_mod2_sparse = csc_matrix(input_train_mod2.X)
input_test_mod2_sparse = csc_matrix(input_test_mod2.X)

In [ ]:
input_train_mod2.X = None
input_test_mod2.X = None
input_train_mod2.X = input_train_mod2_sparse
input_test_mod2.X = input_test_mod2_sparse

In [ ]:
# Write modified ADT datasets
input_train_mod2.write(os.path.join(DATA_DIR,
                                    OUTPUT_SUBTASK_NAME, 
                                    f'{OUTPUT_SUBTASK_NAME}.censor_dataset.output_train_mod2.h5ad'))

input_test_mod2.write(os.path.join(DATA_DIR, 
                                   OUTPUT_SUBTASK_NAME, 
                                   f'{OUTPUT_SUBTASK_NAME}.censor_dataset.output_test_mod2.h5ad'))

In [ ]:
'''
# Copy the raw ADT datasets to the new directory
shutil.copyfile(os.path.join(DATA_DIR, 
                             INPUT_SUBTASK_NAME, 
                             f'{INPUT_SUBTASK_NAME}.censor_dataset.output_train_mod2.h5ad'),
                os.path.join(DATA_DIR, 
                             OUTPUT_SUBTASK_NAME, 
                             f'{OUTPUT_SUBTASK_NAME}.censor_dataset.output_train_mod2.h5ad'))

shutil.copyfile(os.path.join(DATA_DIR, 
                             INPUT_SUBTASK_NAME, 
                             f'{INPUT_SUBTASK_NAME}.censor_dataset.output_test_mod2.h5ad'),
                os.path.join(DATA_DIR, 
                             OUTPUT_SUBTASK_NAME, 
                             f'{OUTPUT_SUBTASK_NAME}.censor_dataset.output_test_mod2.h5ad'))
'''

"\n# Copy the raw ADT datasets to the new directory\nshutil.copyfile(os.path.join(DATA_DIR, \n                             INPUT_SUBTASK_NAME, \n                             f'{INPUT_SUBTASK_NAME}.censor_dataset.output_train_mod2.h5ad'),\n                os.path.join(DATA_DIR, \n                             OUTPUT_SUBTASK_NAME, \n                             f'{OUTPUT_SUBTASK_NAME}.censor_dataset.output_train_mod2.h5ad'))\n\nshutil.copyfile(os.path.join(DATA_DIR, \n                             INPUT_SUBTASK_NAME, \n                             f'{INPUT_SUBTASK_NAME}.censor_dataset.output_test_mod2.h5ad'),\n                os.path.join(DATA_DIR, \n                             OUTPUT_SUBTASK_NAME, \n                             f'{OUTPUT_SUBTASK_NAME}.censor_dataset.output_test_mod2.h5ad'))\n"